In [5]:
########################################
# Import Data & Equations
########################################
import pandas as pd
import os

#define file to import
fname = 'data'
eqfile = fname + '_regression_dataframe_coefficients.csv'
datfile = fname + '.csv'

#Import Data
df = pd.read_csv(datfile, sep=',', header=0)
#Import Equations
eq = pd.read_csv(eqfile, sep=';', header=0)

#clean dataframes
df = df.rename(columns=lambda x: x.strip())
df = df.dropna()
eq = eq.rename(columns=lambda x: x.strip())
eq = eq.fillna('')

#Show Equation Dataframe
print(fname + ' dataset:')
eq

data dataset:


,Group,Muscle,n_features,r2,intercept,ACTIVATION,LRELATIVE,F_V,ForceRelative,Equation
0,DL,BF,3,0.21,57.747855,-36.434307,-41.65571,,25.444838,57.75 + ACTIVATION*-36.43 + LRELATIVE*-41.66 +...
1,DL,SM,3,0.19,312.041921,-35.927431,,-276.42311,8.450483,312.04 + ACTIVATION*-35.93 + F_V*-276.42 + For...
2,DL,ST,3,0.28,-17.334906,-22.482043,17.557921,,-7.74833,-17.33 + ACTIVATION*-22.48 + LRELATIVE*17.56 +...
3,DL,,3,0.18,253.433491,-14.522589,,-233.678485,21.453617,253.43 + ACTIVATION*-14.52 + F_V*-233.68 + For...
4,NHE,BF,3,0.91,-565.443306,-154.032095,,535.537603,162.782768,-565.44 + ACTIVATION*-154.03 + F_V*535.54 + Fo...
5,NHE,SM,3,0.16,-219.645379,-16.504926,,186.105427,72.767172,-219.65 + ACTIVATION*-16.5 + F_V*186.11 + Forc...
6,NHE,ST,3,0.13,505.222769,,-26.675057,-432.542833,4.706382,505.22 + LRELATIVE*-26.68 + F_V*-432.54 + Forc...
7,NHE,,3,0.32,-231.050019,2.870478,36.501049,191.569493,,-231.05 + ACTIVATION*2.87 + LRELATIVE*36.5 + F...
8,,BF,3,0.39,-118.546779,-12.76041,-0.653289,119.563641,,-118.55 + ACTIVATION*-12.76 + LRELATIVE*-0.65 ...
9,,SM,3,0.15,107.633034,6.777107,13.853069,-110.099322,,107.63 + ACTIVATION*6.78 + LRELATIVE*13.85 + F...


In [59]:
###########################
# Validation
###########################
"""
Here we take some random data in dataset to check our model
"""
import numpy as np

#define dependant variable
dep = "SWE_raw"

for alea in range(20):
    #Get an random row among dataframe
    at = np.random.randint(0, df.shape[0])
    mu = df.at[at, 'MUSCLE']
    gr = df.at[at, 'GROUPE']
    
    #get actual dependant value
    y_exp = df.loc[at, dep]
    print(f'{"Actual value for":<20} {dep} at row\t{at:^4} is {y_exp:>8}\t with cat. vars as {gr} & {mu}')
    
    #find appropriate rows in equation dataframe to test MUSCLE, GROUPE & Interaction
    req = eq[(eq['Muscle'] == mu) & (eq['Group'] == gr)]
    req = pd.concat([req, eq[(eq['Muscle'] == mu) & (eq['Group'] == '')]])
    req = pd.concat([req, eq[(eq['Muscle'] == '') & (eq['Group'] == gr)]])
    #define starting column number for indep variables
    scol = 5
    
    #resolve equation for predictions
    for i in req.index:
        y_pred = req.loc[i, 'intercept']
        #loop all columns ad check is there is a value inside, if not, pass to the next one til the end
        for k in range(scol, req.shape[1] - 1):
            #get the colname to get the appropriate value if dataset
            colname = req.columns[k]
            if req.loc[i, colname] == '':
                continue
            x = df.loc[at, colname]
            y_pred += x * req.loc[i, colname]
        #show result of predict
        grp = req.loc[i, 'Group']
        mup =req.loc[i, 'Muscle']
        print(f'{"Predicted value for":<20} {dep} at row\t{at:^4} is {np.around(y_pred, decimals=3):>8}\t with cat. vars as {grp} & {mup}')
    print('---')

Actual value for     SWE_raw at row	 14  is     17.3	 with cat. vars as NHE & ST
Predicted value for  SWE_raw at row	 14  is   14.736	 with cat. vars as NHE & ST
Predicted value for  SWE_raw at row	 14  is    9.816	 with cat. vars as  & ST
Predicted value for  SWE_raw at row	 14  is   11.971	 with cat. vars as NHE & 
---
Actual value for     SWE_raw at row	 48  is     -1.9	 with cat. vars as DL & ST
Predicted value for  SWE_raw at row	 48  is   -1.389	 with cat. vars as DL & ST
Predicted value for  SWE_raw at row	 48  is    2.798	 with cat. vars as  & ST
Predicted value for  SWE_raw at row	 48  is    2.961	 with cat. vars as DL & 
---
Actual value for     SWE_raw at row	 22  is     12.6	 with cat. vars as NHE & BF
Predicted value for  SWE_raw at row	 22  is   10.951	 with cat. vars as NHE & BF
Predicted value for  SWE_raw at row	 22  is    4.671	 with cat. vars as  & BF
Predicted value for  SWE_raw at row	 22  is    3.603	 with cat. vars as NHE & 
---
Actual value for     SWE_raw at ro